In [1]:
import tensorflow as tf
import awkward as ak
import numpy as np
import glob
import os

2021-07-21 11:27:06.343818: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/nvidia-driver/lib64
2021-07-21 11:27:06.343874: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
data_dir = '/eos/cms/store/group/phys_jetmet/dholmber/jec-dnn'
parquet_dir = os.path.join(data_dir, 'preprocessed/dev')

In [3]:
epochs = 10
batch_size = 256
loss = 'mean_absolute_error'
optimizer = 'adam'
lr = 1.e-3

activation = 'relu'
initializer = 'he_normal'
pooling = 'average' # average or max
batch_norm = False
shortcut = False
dropout = 0
K = 16
channels = [
  [64, 64, 64],
  [128, 128, 128],
  [256, 256, 256]
]
units = [128, 128]

train_size = 0.6
test_size = 0.2
val_size = 0.2

num_points = 100

In [4]:
jet_numerical = ['pt_log', 'eta', 'mass', 'phi', 'area', 'qgl_axis2', 'qgl_ptD', 'qgl_mult']
jet_categorical = ['puId', 'partonFlavour']

pf_numerical = ['rel_pt', 'rel_eta', 'rel_phi', 'd0', 'dz', 'd0Err', 'dzErr', 'trkChi2', 'vtxChi2', 'puppiWeight', 'puppiWeightNoLep']
pf_categorical = ['charge', 'lostInnerHits', 'pdgId', 'pvAssocQuality', 'trkQuality']

In [5]:
jet_fields = jet_numerical + jet_categorical
pf_fields = pf_numerical + pf_categorical

jet_keys = [f'jet_{field}' for field in jet_fields]
pf_keys = [f'pf_{field}' for field in pf_fields]

num_jet = len(jet_keys)
num_pf = len(pf_keys)

In [6]:
dirs = glob.glob(os.path.join(parquet_dir, '*'))
num_dirs = len(dirs)
train_split = int(train_size * num_dirs)
test_split = int(test_size * num_dirs) + train_split

train_dirs = dirs[:train_split]
test_dirs = dirs[train_split:test_split]
val_dirs = dirs[test_split:]

In [7]:
train_dirs

['/eos/cms/store/group/phys_jetmet/dholmber/jec-dnn/preprocessed/dev/1',
 '/eos/cms/store/group/phys_jetmet/dholmber/jec-dnn/preprocessed/dev/2',
 '/eos/cms/store/group/phys_jetmet/dholmber/jec-dnn/preprocessed/dev/3']

In [8]:
def read_parquet(path):
    path = path.decode()

    jet = ak.from_parquet(os.path.join(path, 'jet.parquet'))
    pf = ak.from_parquet(os.path.join(path, 'pf.parquet'))
    
    row_lengths = ak.num(pf, axis=1)
    flat_pf = ak.flatten(pf, axis=1)
    
    data = [ak.to_numpy(row_lengths).astype(np.int32), ak.to_numpy(jet.target).astype(np.float32)]
    
    for field in jet_fields:
        data.append(ak.to_numpy(jet[field]).astype(np.float32))

    for field in pf_fields:
        none_padded_pf = ak.pad_none(pf[field], target=num_points, clip=True, axis=1)
        zero_padded_pf = ak.to_numpy(none_padded_pf).filled(0)
        data.append(zero_padded_pf.astype(np.float32))
    
    return data

In [9]:
def read_parquet_wrapper(path):
    inp = [path]
    Tout = [tf.int32] + [tf.float32] + [tf.float32] * num_jet + [tf.float32] * num_pf
    
    cols = tf.numpy_function(read_parquet, inp=inp, Tout=Tout)
    
    keys = ['row_lengths'] + ['target'] + jet_keys + pf_keys
    data = {key: value for key, value in zip(keys, cols)}
    
    data['target'].set_shape((None,))
    
    row_lengths = data.pop('row_lengths')
    row_lengths.set_shape((None,))
    
    for field in jet_keys:
        # Shape from <unknown> to (None,)
        data[field].set_shape((None,))
        # Shape from (None,) to (None, 1)
        data[field] = tf.expand_dims(data[field], axis=1)
    
    for name in pf_keys:
        # Shape from <unknown> to (None, P)
        data[name].set_shape((None, num_points))
        # Shape from (None, P) to (None, P, 1)
        data[name] = tf.expand_dims(data[name], axis=2)
    
    return data

In [10]:
def prepare_pf_inputs(data):
    data['mask'] = tf.cast(tf.math.not_equal(data['pf_rel_eta'], 0), dtype=tf.float32) # 1 if valid
    data['coord_shift'] = tf.multiply(1e6, tf.cast(tf.math.equal(data['mask'], 0), dtype=tf.float32))
    data['points'] = tf.concat([data['pf_rel_eta'], data['pf_rel_phi']], axis=2)
    
    jet_data = tf.concat([data[key] for key in jet_keys], axis=1)
    pf_data = tf.concat([data[key] for key in pf_keys], axis=2)
    
    inputs = (pf_data, jet_data, data['points'], data['coord_shift'], data['mask'])
    return inputs, data['target']

In [11]:
def create_dataset(paths):
    ds = tf.data.Dataset.from_tensor_slices(paths)
    ds = ds.map(read_parquet_wrapper, num_parallel_calls=tf.data.AUTOTUNE)
    ds = ds.map(prepare_pf_inputs, num_parallel_calls=tf.data.AUTOTUNE)
    ds = ds.unbatch().batch(batch_size)
    ds = ds.prefetch(tf.data.AUTOTUNE)
    return ds

In [12]:
train_ds = create_dataset(train_dirs).shuffle(64)
val_ds = create_dataset(val_dirs)
test_ds = create_dataset(test_dirs)

2021-07-21 11:27:09.104941: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2021-07-21 11:27:09.112292: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-07-21 11:27:09.113486: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:00:07.0 name: Tesla V100S-PCIE-32GB computeCapability: 7.0
coreClock: 1.597GHz coreCount: 80 deviceMemorySize: 31.75GiB deviceMemoryBandwidth: 1.03TiB/s
2021-07-21 11:27:09.113636: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/nvidia-driver/lib64
2021-07-21 11:27:09.113722: W tensorflow/stream_executor/platform/default/dso_lo

In [13]:
import tensorflow as tf
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Activation, Add, BatchNormalization, Conv2D, Dense, Dropout, Layer, Multiply, Concatenate
from src.layers import Mean, Max, Expand, Squeeze

In [27]:
def get_particle_net():
    """
    ParticleNet: Jet Tagging via Particle Clouds
    arxiv.org/abs/1902.08570
    
    Parameters
    ----------
    input_shapes : dict
        The shapes of each input (`points`, `features`, `mask`).
    """

    features = Input(name='features', shape=(num_points, num_pf))
    globals = Input(name='globals', shape=(num_jet,))
    points = Input(name='points', shape=(num_points, 2))
    coord_shift = Input(name='coord_shift', shape=(num_points, 1))
    mask = Input(name='mask', shape=(num_points, 1))

    outputs = particle_net_base(points, features, mask, coord_shift, globals)

    model = Model(inputs=[features, globals, points, coord_shift, mask], outputs=outputs)

    model.summary()

    return model


def particle_net_base(points, features, mask, coord_shift, globals):
    """
    points : (N, P, C_coord)
    features:  (N, P, C_features), optional
    mask: (N, P, 1), optional
    """

    # fts = tf.squeeze(BatchNormalization(name='fts_bn')(tf.expand_dims(features, axis=2)), axis=2)
    fts = features
    for layer_idx, sub_channels in enumerate(channels, start=1):
        pts = Add(name=f'add_{layer_idx}')([coord_shift, points]) if layer_idx == 1 else Add(name=f'add_{layer_idx}')([coord_shift, fts])
        fts = edge_conv(
            pts, fts, num_points, sub_channels, name=f'edge_conv_{layer_idx}'
        )

    fts = Multiply()([fts, mask])

    pool = Mean(axis=1)(fts) # (N, C)

    x = Concatenate(name='head')([pool, globals])

    for layer_idx, n in enumerate(units):
        x = Dense(n)(x)
        x = Activation(activation)(x)
        if dropout:
            x = Dropout(dropout)(x)
    out = Dense(1, name='out')(x)
    return out # (N, num_classes)


def edge_conv(points, features, num_points, sub_channels, name):
    """EdgeConv
    Args:
        K: int, number of neighbors
        in_channels: # of input channels
        channels: tuple of output channels
        pooling: pooling method ('max' or 'average')
    Inputs:
        points: (N, P, C_p)
        features: (N, P, C_0)
    Returns:
        transformed points: (N, P, C_out), C_out = channels[-1]
    """

    fts = features
    knn_fts = KNearestNeighbors(num_points, K, name=f'{name}_knn')([points, fts])

    x = knn_fts
    for idx, channel in enumerate(sub_channels, start=1):
        x = Conv2D(
            channel, kernel_size=(1, 1), strides=1, data_format='channels_last',
            use_bias=False if batch_norm else True, kernel_initializer=initializer, name=f'{name}_conv_{idx}'
        )(x)
        if batch_norm:
            x = BatchNormalization(name=f'{name}_batchnorm_{idx}')(x)
        if activation:
            x = Activation(activation, name=f'{name}_activation_{idx}')(x)

    if pooling == 'max':
        fts = Max(axis=2, name=f'{name}_max')(x) # (N, P, C')
    else:
        fts = Mean(axis=2, name=f'{name}_mean')(x) # (N, P, C')

    if shortcut:
        sc = Expand(axis=2, name=f'{name}_shortcut_expand')(features)
        sc = Conv2D(
            sub_channels[-1], kernel_size=(1, 1), strides=1, data_format='channels_last',
            use_bias=False if batch_norm else True, kernel_initializer=initializer, name=f'{name}_shortcut_conv'
        )(sc)
        if batch_norm:
            sc = BatchNormalization(name=f'{name}_shortcut_batchnorm')(sc)
        sc = Squeeze(axis=2, name=f'{name}_shortcut_squeeze')(sc)

        x = Add(name=f'{name}_add')([sc, fts])
    else:
        x = fts

    return Activation(activation, name=f'{name}_activation')(x) # (N, P, C')


class KNearestNeighbors(Layer):
    def __init__(self, num_points, k, **kwargs):
        super().__init__(**kwargs)
        self.num_points = num_points
        self.k = k

    def call(self, inputs):
        points, features = inputs
        # distance
        D = batch_distance_matrix_general(points, points) # (N, P, P)
        _, top_k_indices = tf.math.top_k(-D, k=self.k + 1) # (N, P, K+1)
        top_k_indices = top_k_indices[:, :, 1:] # (N, P, K)

        queries_shape = tf.shape(features)
        batch_size = queries_shape[0]
        batch_indices = tf.tile(tf.reshape(tf.range(batch_size), (-1, 1, 1, 1)), (1, self.num_points, self.k, 1))
        indices = tf.concat([batch_indices, tf.expand_dims(top_k_indices, axis=3)], axis=3) # (N, P, K, 2)
        
        knn_fts =  tf.gather_nd(features, indices) # (N, P, K, C)
        knn_fts_center = tf.tile(tf.expand_dims(features, axis=2), (1, 1, self.k, 1)) # (N, P, K, C)

        return tf.concat([knn_fts_center, tf.subtract(knn_fts, knn_fts_center)], axis=-1) # (N, P, K, 2*C)


# A shape is (N, P_A, C), B shape is (N, P_B, C)
# D shape is (N, P_A, P_B)
def batch_distance_matrix_general(A, B):
    r_A = tf.math.reduce_sum(A * A, axis=2, keepdims=True)
    r_B = tf.math.reduce_sum(B * B, axis=2, keepdims=True)
    m = tf.linalg.matmul(A, tf.transpose(B, perm=(0, 2, 1)))
    D = r_A - 2 * m + tf.transpose(r_B, perm=(0, 2, 1))
    return D

In [28]:
dnn = get_particle_net()
dnn.compile(optimizer=optimizer, loss=loss)
dnn.optimizer.lr.assign(lr)

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
coord_shift (InputLayer)        [(None, 100, 1)]     0                                            
__________________________________________________________________________________________________
points (InputLayer)             [(None, 100, 2)]     0                                            
__________________________________________________________________________________________________
add_1 (Add)                     (None, 100, 2)       0           coord_shift[0][0]                
                                                                 points[0][0]                     
__________________________________________________________________________________________________
features (InputLayer)           [(None, 100, 16)]    0                                      

<tf.Variable 'UnreadVariable' shape=() dtype=float32, numpy=0.001>

In [29]:
# tf.keras.utils.plot_model(dnn, dpi=100, show_shapes=True, expand_nested=True)

In [32]:
fit = dnn.fit(train_ds, validation_data=val_ds, epochs=epochs)

Epoch 1/10
2/2 [==============================] - 20s 10s/step - loss: 11.6469 - val_loss: 10.6499
Epoch 2/10
2/2 [==============================] - 17s 10s/step - loss: 8.7719 - val_loss: 4.3419
Epoch 3/10
2/2 [==============================] - 18s 10s/step - loss: 2.8284 - val_loss: 4.7672
Epoch 4/10
2/2 [==============================] - 18s 10s/step - loss: 5.3100 - val_loss: 0.4959
Epoch 5/10
2/2 [==============================] - 18s 10s/step - loss: 2.2898 - val_loss: 3.0230
Epoch 6/10
2/2 [==============================] - 18s 10s/step - loss: 2.5303 - val_loss: 1.5037
Epoch 7/10
2/2 [==============================] - 18s 10s/step - loss: 2.0474 - val_loss: 1.1952
Epoch 8/10
2/2 [==============================] - 17s 9s/step - loss: 0.7986 - val_loss: 0.7362
Epoch 9/10
2/2 [==============================] - 17s 9s/step - loss: 0.6192 - val_loss: 0.4734
Epoch 10/10
2/2 [==============================] - 17s 9s/step - loss: 0.7848 - val_loss: 0.5175
